### Content Based Recommendations

In the previous notebook, you were introduced to a way to make recommendations using collaborative filtering.  However, using this technique there are a large number of users who were left without any recommendations at all.  Other users were left with fewer than the ten recommendations that were set up by our function to retrieve...

In order to help these users out, let's try another technique **content based** recommendations.  Let's start off where we were in the previous notebook.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import ast
from collections import defaultdict
from IPython.display import HTML
import progressbar
import tests as t
import pickle

In [11]:
# Read in the datasets
movies = pd.read_csv('../data/movies_clean.csv', index_col=0)
reviews = pd.read_csv('../data/reviews_clean.csv', index_col=0)

In [12]:
# This loads our solution dictionary (key: user_id and value: an array of recommended movie titles)
all_recs = {}
with open("data/all_recs.txt", newline="") as csvfile:
    data = csv.reader(csvfile, delimiter=",")
    for row in data:
        val = ast.literal_eval(row[1])
        all_recs[int(row[0])] = val

### Datasets

From the above, you now have access to three important items that you will be using throughout the rest of this notebook.  

`a.` **movies** - a dataframe of all of the movies in the dataset along with other content related information about the movies (genre and date)


`b.` **reviews** - this was the main dataframe used before for collaborative filtering, as it contains all of the interactions between users and movies.


`c.` **all_recs** - a dictionary where each key is a user, and the value is a list of movie recommendations based on collaborative filtering

For the individuals in **all_recs** who did recieve 10 recommendations using collaborative filtering, we don't really need to worry about them.  However, there were a number of individuals in our dataset who did not receive any recommendations.

-----

`1.` To begin, let's start with finding all of the users in our dataset who didn't get all 10 ratings we would have liked them to have using collaborative filtering.  

In [227]:
users_with_all_recs = all_recs.keys()
# Convert `all_recs` dictionary into array of users with all recommendations
rec_number = {user:len(rec) for user, rec in all_recs.items()}
rec_num = {}
rec_num['user_id'] = rec_number.keys()
rec_num['num_recommendations'] = rec_number.values()
rec_num = pd.DataFrame(rec_num)

users_with_all_recs = rec_num['user_id'].drop_duplicates().values
users = reviews['user_id'].drop_duplicates().values

print("There are {} users with all reccomendations from collaborative filtering.".format(len(users_with_all_recs)))

users_who_need_recs = users[~np.isin(users, users_with_all_recs)]

print("There are {} users who still need recommendations.".format(len(users_who_need_recs)))
print("This means that only {}% of users received all 10 of their recommendations using collaborative filtering".format(round(len(users_with_all_recs)/len(np.unique(reviews['user_id'])), 4)*100))   

There are 3503 users with all reccomendations from collaborative filtering.
There are 4519 users who still need recommendations.
This means that only 43.669999999999995% of users received all 10 of their recommendations using collaborative filtering


In [14]:
# Some test here might be nice
assert len(users_who_need_recs) == 4528
print("That's right there were still another 4528 users who needed recommendations when we only used collaborative filtering!")

That's right there were still another 4528 users who needed recommendations when we only used collaborative filtering!


### Content Based Recommendations

You will be doing a bit of a mix of content and collaborative filtering to make recommendations for the users this time.  This will allow you to obtain recommendations in many cases where we didn't make recommendations earlier.     

`2.` Before finding recommendations, rank the user's ratings from highest ratings to lowest ratings. You will move through the movies in this order looking for other similar movies.

In [16]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05 21:00:50
1,2,208092,5,1586466072,2020-04-09 21:01:12
2,2,358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [143]:
rankings = reviews.groupby(['user_id','movie_id']).agg({'rating':'mean'}).sort_values(by=['user_id','rating'], ascending=False)

In [152]:
rankings = rankings.reset_index().set_index('user_id')

### Similarities

In the collaborative filtering sections, you became quite familiar with different methods of determining the similarity (or distance) of two users.  We can perform similarities based on content in much the same way.  

In many cases, it turns out that one of the fastest ways we can find out how similar items are to one another (when our matrix isn't totally sparse like it was in the earlier section) is by simply using matrix multiplication.  If you are not familiar with this, an explanation is available [here by 3blue1brown](https://www.youtube.com/watch?v=LyGKycYT2v0) and another quick explanation is provided [on the post here](https://math.stackexchange.com/questions/689022/how-does-the-dot-product-determine-similarity).

For us to pull out a matrix that describes the movies in our dataframe in terms of content, we might just use the indicator variables related to **year** and **genre** for our movies.  

Then we can obtain a matrix of how similar movies are to one another by taking the dot product of this matrix with itself.  Notice in the below that the dot product where our 1 values overlap gives a value of 2 indicating higher similarity.  In the second dot product, the 1 values don't match up.  This leads to a dot product of 0 indicating lower similarity.

<img src="images/dotprod1.png" alt="Dot Product" height="500" width="500">

We can perform the dot product on a matrix of movies with content characteristics to provide a movie by movie matrix where each cell is an indication of how similar two movies are to one another.  In the below image, you can see that movies 1 and 8 are most similar, movies 2 and 8 are most similar and movies 3 and 9 are most similar for this subset of the data.  The diagonal elements of the matrix will contain the similarity of a movie with itself, which will be the largest possible similarity (which will also be the number of 1's in the movie row within the orginal movie content matrix.

<img src="images/moviemat.png" alt="Dot Product" height="500" width="500">


`3.` Create a numpy array that is a matrix of indicator variables related to year (by century) and movie genres by movie.  Perform the dot prodoct of this matrix with itself (transposed) to obtain a similarity matrix of each movie with every other movie.  The final matrix should be 31245 x 31245.

In [25]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Documentary,Adventure,Animation,...,War,Romance,Musical,Film-Noir,Sport,Mystery,Western,Horror,Comedy,Music
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [45]:
M = movies.drop(['movie_id','movie','genre','date'], axis=1).values

In [46]:
M.shape

(35479, 31)

In [47]:
sim_matrix = np.dot(M, M.T)

In [172]:
sim_matrix

array([[3, 3, 3, ..., 1, 1, 0],
       [3, 3, 3, ..., 1, 1, 0],
       [3, 3, 3, ..., 1, 1, 0],
       ...,
       [1, 1, 1, ..., 2, 2, 1],
       [1, 1, 1, ..., 2, 2, 1],
       [0, 0, 0, ..., 1, 1, 1]], dtype=int64)

In [48]:
# create checks for the dot product matrix
assert sim_matrix.shape[0] == 35479
assert sim_matrix.shape[1] == 35479
assert sim_matrix[0, 0] == np.max(sim_matrix[0])
print("Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!")

Looks like you passed all of the tests.  Though they weren't very robust - if you want to write some of your own, I won't complain!


### For Each User...


Now that you have a matrix where each user has their ratings ordered.  You also have a second matrix where movies are each axis, and the matrix entries are larger where the two movies are more similar and smaller where the two movies are dissimilar.  This matrix is a measure of content similarity. Therefore, it is time to get to the fun part.

For each user, we will perform the following:

    i. For each movie, find the movies that are most similar that the user hasn't seen.

    ii. Continue through the available, rated movies until 10 recommendations or until there are no additional movies.

As a final note, you may need to adjust the criteria for 'most similar' to obtain 10 recommendations.  As a first pass, I used only movies with the highest possible similarity to one another as similar enough to add as a recommendation.

`3.` In the below cell, complete each of the functions needed for making content based recommendations.

In [63]:
M_argsort  = np.argsort(sim_matrix,axis=1)

In [108]:
M_argsort[0][:-1].shape

(35478,)

In [109]:
M_argsort

array([[17739, 22184, 22183, ..., 13515,     1,     0],
       [17739, 22184, 22183, ..., 13515,     1,     0],
       [17739, 22184, 22183, ..., 13515,     1,     0],
       ...,
       [22153,  7357,  7358, ..., 15716, 29697, 22151],
       [22153,  7357,  7358, ..., 15716, 29697, 22151],
       [    0,  7479,  7480, ..., 19228, 19237, 35478]], dtype=int64)

In [113]:
most_similar_movies = M_argsort

In [119]:
most_similar_movies

array([[17739, 22184, 22183, ..., 13515,     1,     0],
       [17739, 22184, 22183, ..., 13515,     1,     0],
       [17739, 22184, 22183, ..., 13515,     1,     0],
       ...,
       [22153,  7357,  7358, ..., 15716, 29697, 22151],
       [22153,  7357,  7358, ..., 15716, 29697, 22151],
       [    0,  7479,  7480, ..., 19228, 19237, 35478]], dtype=int64)

In [135]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Documentary,Adventure,Animation,...,War,Romance,Musical,Film-Noir,Sport,Mystery,Western,Horror,Comedy,Music
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [175]:
movie_id=10

ind = movies[movies['movie_id']==movie_id].index[0]
similar_movies_index = most_similar_movies[ind][~np.isin(most_similar_movies[ind],ind)][-2:]
movie_ids = similar_movies_index[::-1]
similar_movies = get_movie_names(movie_ids)

similar_movies

array(['Edison Kinetoscopic Record of a Sneeze (1894)',
       'Roundhay Garden Scene (1888)'], dtype=object)

In [260]:
def find_similar_movies(movie_id):
    '''
    INPUT
    movie_id - a movie_id 
    OUTPUT
    similar_movies - an array of the most similar movies by title
    '''
    try:
        ind = movies[movies['movie_id']==movie_id].index[0]
        similar_movies_index = most_similar_movies[ind][~np.isin(most_similar_movies[ind],ind)][-10:]
        movie_ids = similar_movies_index[::-1]
        similar_movies = get_movie_names(movie_ids)
        
        return similar_movies
    except:
        pass
    
    
def get_movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = np.array(movies.iloc[movie_ids]['movie'])
    
    return movie_lst

def make_recs():
    '''
    INPUT
    None
    OUTPUT
    recs - a dictionary with keys of the user and values of the recommendations
    '''
    recs = {}
    users = rankings.index.unique()
    for user in users:
        movie_ids = np.array(rankings.loc[user][['movie_id']])
        for i in range(len(movie_ids)):
            try:
                if find_similar_movies(movie_ids[i]) is not None:
                    recs[user] = find_similar_movies(movie_ids[i])
                    break
                else:
                    pass
            except:
                pass
            
    return recs

In [261]:
recs = make_recs()

### How Did We Do?

Now that you have made the recommendations, how did we do in providing everyone with a set of recommendations?

`4.` Use the cells below to see how many individuals you were able to make recommendations for, as well as explore characteristics about individuals who you were not able to make recommendations for.  

In [219]:
rec_number = {user:len(rec) for user, rec in recs.items()}
rec_num = {}
rec_num['user_id'] = rec_number.keys()
rec_num['num_recommendations'] = rec_number.values()
rec_num2 = pd.DataFrame(rec_num)

In [220]:
rec_num2

,user_id,num_recommendations
0,8021,10
1,8019,10
2,8017,10
3,8016,10
4,8015,10
...,...,...
3498,15,10
3499,14,10
3500,12,10
3501,10,10


In [226]:
rec_num2[rec_num2['num_recommendations']<10]

,user_id,num_recommendations


In [221]:
rec_number = {user:len(rec) for user, rec in all_recs.items()}
rec_num = {}
rec_num['user_id'] = rec_number.keys()
rec_num['num_recommendations'] = rec_number.values()
rec_num1 = pd.DataFrame(rec_num)

In [224]:
rec_num1[rec_num1['num_recommendations']<10]

,user_id,num_recommendations


In [200]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [210]:
movie_ids = user_by_movie.loc[2].dropna().index

In [214]:
np.array(movies['movie'][np.isin(movies['movie_id'],movie_ids)])

array(['Snatch (2000)', 'Walk the Line (2005)', 'Gisaengchung (2019)',
       'Once Upon a Time ...in Hollywood (2019)',
       'Jumanji: The Next Level (2019)', 'The Lighthouse (2019)',
       'The Gentlemen (2019)', '1917 (2019)', 'Knives Out (2019)',
       'Countdown (2019)'], dtype=object)

In [216]:
# Closer look at individual user characteristics
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movie_ids = user_by_movie.loc[user_id].dropna().index
    movie_names = np.array(movies['movie'][np.isin(movies['movie_id'],movie_ids)])
    
    return movie_names

movies_watched(189)

array(['Prisoners (2013)', 'Gravity (2013)'], dtype=object)

In [263]:
with_recs = np.unique(list(all_recs.keys()) + list(recs.keys()))
all_users = reviews['user_id'].unique()

In [264]:
no_recs = reviews.loc[~np.isin(reviews['user_id'], with_recs)]['user_id'].unique()

In [266]:
no_recs.shape

(1025,)

In [269]:
cnter = 0
print("Some of the movie lists for users without any recommendations include:")
for user_id in no_recs:
    print(user_id)
    print(movies_watched(user_id))
    cnter+=1
    if cnter > 10:
        break

Some of the movie lists for users without any recommendations include:
9
['The Italian Job (2003)' 'Mommy (2014)']
37
['Her (2013)' 'The Hangover Part III (2013)']
50
['Friends with Benefits (2011)' 'Avengers: Endgame (2019)']
66
['Black Dynamite (2009)' 'Despicable Me 2 (2013)']
78
['Stoker (2013)' "La vie d'Adèle (2013)"]
80
['Interstellar (2014)' 'The Irishman (2019)']
111
['Kick-Ass 2 (2013)' 'Saving Mr. Banks (2013)']
130
['The Fifth Element (1997)' 'Tyrannosaur (2011)']
136
['Halloween (1978)' 'Yesterday (2019)']
151
['The Prestige (2006)' 'Inception (2010)']
160
['Cool Hand Luke (1967)' 'In the Heat of the Night (1967)']


### Now What?  

Well, if you were really strict with your criteria for how similar two movies (like I was initially), then you still have some users that don't have all 10 recommendations (and a small group of users who have no recommendations at all). 

As stated earlier, recommendation engines are a bit of an **art** and a **science**.  There are a number of things we still could look into - how do our collaborative filtering and content based recommendations compare to one another? How could we incorporate user input along with collaborative filtering and/or content based recommendations to improve any of our recommendations?  How can we truly gain recommendations for every user?

`5.` In this last step feel free to explore any last ideas you have with the recommendation techniques we have looked at so far.  You might choose to make the final needed recommendations using the first technique with just top ranked movies.  You might also loosen up the strictness in the similarity needed between movies.  Be creative and share your insights with your classmates!